In [8]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, clear_output

In [ ]:
# Define the standard functions and their properties
# Each function: (name, function, inverse_function_if_exists, domain, properties)

def get_function_definition(func_type, a, b, c, h_shift, v_shift, h_scale, v_scale):
    """
    Get function and its properties based on type and parameters.
    
    Parameters:
    - func_type: type of function (linear, quadratic, etc.)
    - a, b, c: function-specific parameters
    - h_shift, v_shift: horizontal and vertical translation
    - h_scale, v_scale: horizontal and vertical scaling
    
    Returns: (func, inverse_func, domain, is_monotonic, is_symmetric, is_convex, is_concave, is_nonnegative)
    """
    
    # Base functions before transformation
    # Transformation: y = v_scale * f((x - h_shift) / h_scale) + v_shift
    
    if func_type == "Linear":
        # f(x) = ax + b
        def base_func(x):
            return a * x + b
        def func(x):
            t = (x - h_shift) / h_scale
            return v_scale * base_func(t) + v_shift
        
        # Inverse exists if a != 0
        if a != 0:
            def inverse_func(y):
                # y = v_scale * (a * ((x - h_shift)/h_scale) + b) + v_shift
                # Solve for x:
                # (y - v_shift) / v_scale = a * ((x - h_shift)/h_scale) + b
                # ((y - v_shift) / v_scale - b) / a = (x - h_shift) / h_scale
                # x = h_scale * ((y - v_shift) / v_scale - b) / a + h_shift
                return h_scale * (((y - v_shift) / v_scale - b) / a) + h_shift
        else:
            inverse_func = None
        
        domain = (-10, 10)
        is_monotonic = (a != 0)
        is_symmetric = False
        is_convex = True  # Linear is both convex and concave
        is_concave = True
        is_nonnegative = False  # Generally not
        
    elif func_type == "Quadratic":
        # f(x) = ax^2 + bx + c
        def base_func(x):
            return a * x**2 + b * x + c
        def func(x):
            t = (x - h_shift) / h_scale
            return v_scale * base_func(t) + v_shift
        
        inverse_func = None  # Not monotonic in general
        domain = (-10, 10)
        is_monotonic = False
        is_symmetric = (b == 0)  # Symmetric about y-axis when b=0
        is_convex = (a * v_scale > 0)
        is_concave = (a * v_scale < 0)
        # Nonnegative check is complex, simplified
        is_nonnegative = False
        
    elif func_type == "Cubic":
        # f(x) = ax^3 + bx^2 + cx
        def base_func(x):
            return a * x**3 + b * x**2 + c * x
        def func(x):
            t = (x - h_shift) / h_scale
            return v_scale * base_func(t) + v_shift
        
        # Only monotonic when a > 0 and b^2 <= 3ac (no local extrema)
        # For simplicity, check if purely cubic (b=0, c=0)
        if b == 0 and c == 0 and a != 0:
            def inverse_func(y):
                # y = v_scale * a * ((x-h_shift)/h_scale)^3 + v_shift
                inner = (y - v_shift) / (v_scale * a)
                return h_scale * np.sign(inner) * np.abs(inner)**(1/3) + h_shift
            is_monotonic = True
        else:
            inverse_func = None
            is_monotonic = False
        
        domain = (-10, 10)
        is_symmetric = (b == 0)  # Odd function when b=0
        is_convex = False
        is_concave = False
        is_nonnegative = False
        
    elif func_type == "Power":
        # f(x) = x^a (defined for x > 0)
        def base_func(x):
            return np.power(np.maximum(x, 1e-10), a)
        def func(x):
            t = (x - h_shift) / h_scale
            return v_scale * base_func(t) + v_shift
        
        # Inverse: x = y^(1/a) for y > 0
        if a != 0:
            def inverse_func(y):
                inner = (y - v_shift) / v_scale
                inner = np.maximum(inner, 1e-10)
                return h_scale * np.power(inner, 1/a) + h_shift
            is_monotonic = True
        else:
            inverse_func = None
            is_monotonic = False
        
        domain = (0.01, 10)
        is_symmetric = False
        is_convex = (a >= 1 or a < 0) and (a * v_scale > 0)
        is_concave = (0 < a < 1) and (v_scale > 0)
        is_nonnegative = (v_shift >= 0) and (v_scale >= 0)
        
    elif func_type == "Root":
        # f(x) = x^(1/a) = a-th root (defined for x >= 0)
        def base_func(x):
            return np.power(np.maximum(x, 0), 1/a)
        def func(x):
            t = (x - h_shift) / h_scale
            return v_scale * base_func(t) + v_shift
        
        # Inverse: x = y^a
        if a != 0:
            def inverse_func(y):
                inner = (y - v_shift) / v_scale
                return h_scale * np.power(np.maximum(inner, 0), a) + h_shift
            is_monotonic = True
        else:
            inverse_func = None
            is_monotonic = False
        
        domain = (0, 10)
        is_symmetric = False
        is_convex = False
        is_concave = (a > 0) and (v_scale > 0)
        is_nonnegative = (v_shift >= 0) and (v_scale >= 0)
        
    elif func_type == "Exponential":
        # f(x) = a * b^x (base b > 0, b != 1)
        base = max(b, 0.1)  # Ensure positive base
        def base_func(x):
            return a * np.power(base, x)
        def func(x):
            t = (x - h_shift) / h_scale
            return v_scale * base_func(t) + v_shift
        
        # Inverse: x = log_b(y/a)
        if a != 0 and base > 0 and base != 1:
            def inverse_func(y):
                inner = (y - v_shift) / (v_scale * a)
                inner = np.maximum(inner, 1e-10)
                return h_scale * np.log(inner) / np.log(base) + h_shift
            is_monotonic = True
        else:
            inverse_func = None
            is_monotonic = False
        
        domain = (-5, 5)
        is_symmetric = False
        is_convex = (a * v_scale > 0)
        is_concave = (a * v_scale < 0)
        is_nonnegative = (a > 0) and (v_scale >= 0) and (v_shift >= 0)
        
    elif func_type == "Logarithm":
        # f(x) = a * log_b(x) (base b > 0, b != 1, x > 0)
        base = max(b, 0.1)
        if base == 1:
            base = 2  # Avoid log base 1
        def base_func(x):
            return a * np.log(np.maximum(x, 1e-10)) / np.log(base)
        def func(x):
            t = (x - h_shift) / h_scale
            return v_scale * base_func(t) + v_shift
        
        # Inverse: x = b^(y/a)
        if a != 0:
            def inverse_func(y):
                inner = (y - v_shift) / (v_scale * a)
                return h_scale * np.power(base, inner) + h_shift
            is_monotonic = True
        else:
            inverse_func = None
            is_monotonic = False
        
        domain = (0.01, 10)
        is_symmetric = False
        is_convex = (a * v_scale < 0)
        is_concave = (a * v_scale > 0)
        is_nonnegative = False
    
    else:
        raise ValueError(f"Unknown function type: {func_type}")
    
    return func, inverse_func, domain, is_monotonic, is_symmetric, is_convex, is_concave, is_nonnegative

In [ ]:
class FunctionExplorerVisualization:
    """Interactive visualization for exploring standard functions and their properties"""
    
    def __init__(self):
        self.plot_output = widgets.Output()
        self.show_inverse = False
        
        # Function types
        self.function_types = ["Linear", "Quadratic", "Cubic", "Power", "Root", "Exponential", "Logarithm"]
        
        self._create_widgets()
        self._setup_callbacks()
        
    def _create_widgets(self):
        """Create all widgets"""
        
        # Function type dropdown
        self.func_dropdown = widgets.Dropdown(
            options=self.function_types,
            value="Linear",
            description="Function:",
            style={'description_width': 'initial'}
        )
        
        # Function-specific parameters
        self.param_a = widgets.FloatSlider(
            value=1, min=-5, max=5, step=0.1,
            description='a:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.param_b = widgets.FloatSlider(
            value=0, min=-5, max=5, step=0.1,
            description='b:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.param_c = widgets.FloatSlider(
            value=0, min=-5, max=5, step=0.1,
            description='c:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        # Parameter description label
        self.param_description = widgets.HTML(
            value='<b>f(x) = ax + b</b>',
            layout=widgets.Layout(margin='5px 0')
        )
        
        # Transformation sliders
        self.h_shift = widgets.FloatSlider(
            value=0, min=-5, max=5, step=0.1,
            description='H-Shift:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.v_shift = widgets.FloatSlider(
            value=0, min=-5, max=5, step=0.1,
            description='V-Shift:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.h_scale = widgets.FloatSlider(
            value=1, min=0.1, max=5, step=0.1,
            description='H-Scale:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.v_scale = widgets.FloatSlider(
            value=1, min=-5, max=5, step=0.1,
            description='V-Scale:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        # Cursor position slider
        self.cursor_slider = widgets.FloatSlider(
            value=1, min=-5, max=5, step=0.05,
            description='Cursor x:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        # Properties checkboxes (for quiz)
        self.check_symmetric = widgets.Checkbox(
            value=False,
            description='Symmetric (even function)',
            style={'description_width': 'initial'},
            indent=False
        )
        
        self.check_monotonic = widgets.Checkbox(
            value=False,
            description='Monotonic',
            style={'description_width': 'initial'},
            indent=False
        )
        
        self.check_convex = widgets.Checkbox(
            value=False,
            description='Convex',
            style={'description_width': 'initial'},
            indent=False
        )
        
        self.check_concave = widgets.Checkbox(
            value=False,
            description='Concave',
            style={'description_width': 'initial'},
            indent=False
        )
        
        self.check_nonnegative = widgets.Checkbox(
            value=False,
            description='Nonnegative',
            style={'description_width': 'initial'},
            indent=False
        )
        
        # Check answers button
        self.check_button = widgets.Button(
            description="Check Answers",
            button_style='primary',
            layout=widgets.Layout(width='150px')
        )
        
        # Reset button
        self.reset_button = widgets.Button(
            description="Reset",
            button_style='warning',
            layout=widgets.Layout(width='100px')
        )
        
        # Show inverse button (will be enabled when function is monotonic)
        self.inverse_button = widgets.ToggleButton(
            value=False,
            description='Show Inverse',
            button_style='info',
            disabled=True,
            layout=widgets.Layout(width='150px')
        )
        
        # Quiz feedback
        self.quiz_feedback = widgets.HTML(
            value='<div style="padding: 10px; background-color: #f5f5f5; border-radius: 5px;">Select properties and click "Check Answers"</div>'
        )
        
        # Cursor info display
        self.cursor_info = widgets.HTML(
            value='<div style="padding: 5px; font-family: monospace;"></div>'
        )
        
    def _setup_callbacks(self):
        """Setup widget callbacks"""
        self.func_dropdown.observe(self._on_func_change, names='value')
        
        # All parameter sliders update the plot
        for slider in [self.param_a, self.param_b, self.param_c, 
                       self.h_shift, self.v_shift, self.h_scale, self.v_scale,
                       self.cursor_slider]:
            slider.observe(self._on_param_change, names='value')
        
        self.check_button.on_click(self._on_check_answers)
        self.reset_button.on_click(self._on_reset)
        self.inverse_button.observe(self._on_inverse_toggle, names='value')
        
    def _update_param_visibility(self):
        """Update parameter visibility and descriptions based on function type"""
        func_type = self.func_dropdown.value
        
        if func_type == "Linear":
            self.param_description.value = '<b>f(x) = ax + b</b>'
            self.param_a.description = 'a (slope):'
            self.param_b.description = 'b (intercept):'
            self.param_a.min, self.param_a.max = -5, 5
            self.param_b.min, self.param_b.max = -5, 5
            self.param_c.layout.visibility = 'hidden'
            self.param_b.layout.visibility = 'visible'
            
        elif func_type == "Quadratic":
            self.param_description.value = '<b>f(x) = ax² + bx + c</b>'
            self.param_a.description = 'a:'
            self.param_b.description = 'b:'
            self.param_c.description = 'c:'
            self.param_a.min, self.param_a.max = -5, 5
            self.param_b.min, self.param_b.max = -5, 5
            self.param_c.layout.visibility = 'visible'
            self.param_b.layout.visibility = 'visible'
            
        elif func_type == "Cubic":
            self.param_description.value = '<b>f(x) = ax³ + bx² + cx</b>'
            self.param_a.description = 'a:'
            self.param_b.description = 'b:'
            self.param_c.description = 'c:'
            self.param_a.min, self.param_a.max = -5, 5
            self.param_b.min, self.param_b.max = -5, 5
            self.param_c.layout.visibility = 'visible'
            self.param_b.layout.visibility = 'visible'
            
        elif func_type == "Power":
            self.param_description.value = '<b>f(x) = x<sup>a</sup></b> (x > 0)'
            self.param_a.description = 'a (exponent):'
            self.param_a.min, self.param_a.max = -3, 5
            self.param_b.layout.visibility = 'hidden'
            self.param_c.layout.visibility = 'hidden'
            
        elif func_type == "Root":
            self.param_description.value = '<b>f(x) = x<sup>1/a</sup> = ᵃ√x</b> (x ≥ 0)'
            self.param_a.description = 'a (root):'
            self.param_a.min, self.param_a.max = 1, 10
            if self.param_a.value < 1:
                self.param_a.value = 2
            self.param_b.layout.visibility = 'hidden'
            self.param_c.layout.visibility = 'hidden'
            
        elif func_type == "Exponential":
            self.param_description.value = '<b>f(x) = a · b<sup>x</sup></b>'
            self.param_a.description = 'a (scale):'
            self.param_b.description = 'b (base):'
            self.param_a.min, self.param_a.max = -5, 5
            self.param_b.min, self.param_b.max = 0.1, 5
            if self.param_b.value <= 0:
                self.param_b.value = 2
            self.param_b.layout.visibility = 'visible'
            self.param_c.layout.visibility = 'hidden'
            
        elif func_type == "Logarithm":
            self.param_description.value = '<b>f(x) = a · log<sub>b</sub>(x)</b> (x > 0)'
            self.param_a.description = 'a (scale):'
            self.param_b.description = 'b (base):'
            self.param_a.min, self.param_a.max = -5, 5
            self.param_b.min, self.param_b.max = 0.1, 10
            if self.param_b.value <= 0 or self.param_b.value == 1:
                self.param_b.value = 2
            self.param_b.layout.visibility = 'visible'
            self.param_c.layout.visibility = 'hidden'
    
    def _on_func_change(self, change):
        """Handle function type change"""
        self._update_param_visibility()
        self._reset_checkboxes()
        self._update_inverse_button()
        self._update_plot()
        
    def _on_param_change(self, change):
        """Handle parameter changes"""
        self._update_inverse_button()
        self._update_plot()
        
    def _on_inverse_toggle(self, change):
        """Handle inverse toggle"""
        self.show_inverse = change['new']
        self._update_plot()
        
    def _reset_checkboxes(self):
        """Reset all property checkboxes"""
        self.check_symmetric.value = False
        self.check_monotonic.value = False
        self.check_convex.value = False
        self.check_concave.value = False
        self.check_nonnegative.value = False
        self.quiz_feedback.value = '<div style="padding: 10px; background-color: #f5f5f5; border-radius: 5px;">Select properties and click "Check Answers"</div>'
        
    def _on_reset(self, button):
        """Reset all parameters and checkboxes"""
        self.param_a.value = 1
        self.param_b.value = 0
        self.param_c.value = 0
        self.h_shift.value = 0
        self.v_shift.value = 0
        self.h_scale.value = 1
        self.v_scale.value = 1
        self.cursor_slider.value = 1
        self.inverse_button.value = False
        self._reset_checkboxes()
        self._update_plot()
        
    def _update_inverse_button(self):
        """Enable/disable inverse button based on whether function is monotonic"""
        func, inv_func, domain, is_monotonic, _, _, _, _ = get_function_definition(
            self.func_dropdown.value,
            self.param_a.value, self.param_b.value, self.param_c.value,
            self.h_shift.value, self.v_shift.value,
            self.h_scale.value, self.v_scale.value
        )
        
        if is_monotonic and inv_func is not None:
            self.inverse_button.disabled = False
            self.inverse_button.button_style = 'info'
        else:
            self.inverse_button.disabled = True
            self.inverse_button.value = False
            self.show_inverse = False
            self.inverse_button.button_style = ''
    
    def _on_check_answers(self, button):
        """Check the user's property answers"""
        func, inv_func, domain, is_monotonic, is_symmetric, is_convex, is_concave, is_nonnegative = get_function_definition(
            self.func_dropdown.value,
            self.param_a.value, self.param_b.value, self.param_c.value,
            self.h_shift.value, self.v_shift.value,
            self.h_scale.value, self.v_scale.value
        )
        
        # Check each answer
        results = []
        correct_count = 0
        total = 5
        
        # Symmetric
        if self.check_symmetric.value == is_symmetric:
            results.append(('Symmetric', '✓', 'green'))
            correct_count += 1
        else:
            results.append(('Symmetric', '✗', 'red'))
            
        # Monotonic
        if self.check_monotonic.value == is_monotonic:
            results.append(('Monotonic', '✓', 'green'))
            correct_count += 1
        else:
            results.append(('Monotonic', '✗', 'red'))
            
        # Convex
        if self.check_convex.value == is_convex:
            results.append(('Convex', '✓', 'green'))
            correct_count += 1
        else:
            results.append(('Convex', '✗', 'red'))
            
        # Concave
        if self.check_concave.value == is_concave:
            results.append(('Concave', '✓', 'green'))
            correct_count += 1
        else:
            results.append(('Concave', '✗', 'red'))
            
        # Nonnegative
        if self.check_nonnegative.value == is_nonnegative:
            results.append(('Nonnegative', '✓', 'green'))
            correct_count += 1
        else:
            results.append(('Nonnegative', '✗', 'red'))
        
        # Build feedback HTML
        feedback_html = f'<div style="padding: 10px; background-color: {"#d4edda" if correct_count == total else "#f8d7da"}; border-radius: 5px;">'
        feedback_html += f'<b>Score: {correct_count}/{total}</b><br><br>'
        for name, symbol, color in results:
            feedback_html += f'<span style="color: {color};">{symbol} {name}</span><br>'
        
        # Add correct answers
        feedback_html += '<br><b>Correct answers:</b><br>'
        feedback_html += f'Symmetric: {"Yes" if is_symmetric else "No"}<br>'
        feedback_html += f'Monotonic: {"Yes" if is_monotonic else "No"}<br>'
        feedback_html += f'Convex: {"Yes" if is_convex else "No"}<br>'
        feedback_html += f'Concave: {"Yes" if is_concave else "No"}<br>'
        feedback_html += f'Nonnegative: {"Yes" if is_nonnegative else "No"}'
        feedback_html += '</div>'
        
        self.quiz_feedback.value = feedback_html
        
    def _update_plot(self):
        """Update the plot with current function and cursor"""
        with self.plot_output:
            clear_output(wait=True)
            
            func_type = self.func_dropdown.value
            a, b, c = self.param_a.value, self.param_b.value, self.param_c.value
            h_shift, v_shift = self.h_shift.value, self.v_shift.value
            h_scale, v_scale = self.h_scale.value, self.v_scale.value
            
            func, inv_func, domain, is_monotonic, _, _, _, _ = get_function_definition(
                func_type, a, b, c, h_shift, v_shift, h_scale, v_scale
            )
            
            # Create figure
            fig = go.Figure()
            
            # Compute x range based on domain and transformations
            x_min = h_scale * domain[0] + h_shift
            x_max = h_scale * domain[1] + h_shift
            
            # Adjust for viewing - use symmetric range for square plot
            plot_x_min = -10
            plot_x_max = 10
            plot_y_min = -10
            plot_y_max = 10
            
            x = np.linspace(max(x_min, plot_x_min), min(x_max, plot_x_max), 1000)
            
            # Compute function values
            with np.errstate(all='ignore'):
                y = func(x)
                # Replace inf/nan with nan for plotting
                y = np.where(np.isfinite(y), y, np.nan)
            
            # Plot the main function
            fig.add_trace(go.Scatter(
                x=x, y=y,
                mode='lines',
                name=f'f(x)',
                line=dict(color='blue', width=3)
            ))
            
            # Handle cursor position
            cursor_x = self.cursor_slider.value
            
            # Make sure cursor is in domain
            if cursor_x >= x_min and cursor_x <= x_max:
                with np.errstate(all='ignore'):
                    cursor_y = func(cursor_x)
                
                if np.isfinite(cursor_y):
                    # Plot cursor point on the function
                    fig.add_trace(go.Scatter(
                        x=[cursor_x], y=[cursor_y],
                        mode='markers',
                        name=f'(x, f(x)) = ({cursor_x:.2f}, {cursor_y:.2f})',
                        marker=dict(color='blue', size=15, symbol='circle',
                                  line=dict(color='white', width=2))
                    ))
                    
                    # Update cursor info
                    self.cursor_info.value = f'<div style="padding: 5px; font-family: monospace; background-color: #e3f2fd; border-radius: 3px;"><b>Cursor:</b> x = {cursor_x:.3f}, f(x) = {cursor_y:.3f}</div>'
                    
                    # If showing inverse
                    if self.show_inverse and inv_func is not None:
                        # Plot x = y line
                        diag_range = np.linspace(-15, 15, 100)
                        fig.add_trace(go.Scatter(
                            x=diag_range, y=diag_range,
                            mode='lines',
                            name='y = x (diagonal)',
                            line=dict(color='gray', width=2, dash='dash')
                        ))
                        
                        # Compute inverse function values
                        # For the inverse, x and y are swapped
                        y_range = np.linspace(np.nanmin(y[np.isfinite(y)]), np.nanmax(y[np.isfinite(y)]), 500)
                        
                        with np.errstate(all='ignore'):
                            inv_x = inv_func(y_range)
                            inv_x = np.where(np.isfinite(inv_x), inv_x, np.nan)
                        
                        # Plot inverse function (swapped x and y)
                        fig.add_trace(go.Scatter(
                            x=y_range, y=inv_x,
                            mode='lines',
                            name='f⁻¹(x)',
                            line=dict(color='red', width=3)
                        ))
                        
                        # The inverse point: (f(x), x)
                        inv_point_x = cursor_y
                        inv_point_y = cursor_x
                        
                        # Point on the diagonal: midpoint of reflection
                        # The midpoint between (cursor_x, cursor_y) and (cursor_y, cursor_x)
                        diag_x = (cursor_x + cursor_y) / 2
                        diag_y = (cursor_x + cursor_y) / 2
                        
                        # Draw the shaded square with corners:
                        # (x, f(x)) - cursor position on f
                        # (x, x) - on the diagonal below/above cursor
                        # (f(x), f(x)) - on the diagonal 
                        # (f(x), x) - inverse point
                        
                        square_x = [cursor_x, cursor_x, cursor_y, cursor_y, cursor_x]
                        square_y = [cursor_y, cursor_x, cursor_x, cursor_y, cursor_y]
                        
                        fig.add_trace(go.Scatter(
                            x=square_x, y=square_y,
                            mode='lines',
                            name='Reflection square',
                            fill='toself',
                            fillcolor='rgba(255, 200, 100, 0.3)',
                            line=dict(color='orange', width=2, dash='dot')
                        ))
                        
                        # Mark the diagonal points
                        fig.add_trace(go.Scatter(
                            x=[cursor_x, cursor_y],
                            y=[cursor_x, cursor_y],
                            mode='markers',
                            name='Points on y=x',
                            marker=dict(color='gray', size=10, symbol='diamond')
                        ))
                        
                        # Mark the inverse point
                        fig.add_trace(go.Scatter(
                            x=[inv_point_x], y=[inv_point_y],
                            mode='markers',
                            name=f'(f(x), x) = ({inv_point_x:.2f}, {inv_point_y:.2f})',
                            marker=dict(color='red', size=15, symbol='circle',
                                      line=dict(color='white', width=2))
                        ))
                        
                        # Update cursor info with inverse
                        self.cursor_info.value = (
                            f'<div style="padding: 5px; font-family: monospace; background-color: #e3f2fd; border-radius: 3px;">'
                            f'<b>Original:</b> (x, f(x)) = ({cursor_x:.3f}, {cursor_y:.3f})<br>'
                            f'<b>Inverse:</b> (f(x), x) = ({cursor_y:.3f}, {cursor_x:.3f})'
                            f'</div>'
                        )
                else:
                    self.cursor_info.value = '<div style="padding: 5px; font-family: monospace; color: red;">Cursor position outside function domain</div>'
            else:
                self.cursor_info.value = '<div style="padding: 5px; font-family: monospace; color: red;">Cursor position outside function domain</div>'
            
            # Set axis properties
            fig.update_layout(
                title=f'{func_type} Function Explorer',
                xaxis_title='x',
                yaxis_title='y',
                width=650,
                height=650,
                xaxis=dict(range=[plot_x_min, plot_x_max], zeroline=True, zerolinewidth=1, zerolinecolor='black', constrain='domain'),
                yaxis=dict(range=[plot_y_min, plot_y_max], zeroline=True, zerolinewidth=1, zerolinecolor='black', scaleanchor='x', scaleratio=1, constrain='domain'),
                showlegend=True,
                legend=dict(x=1.02, y=1, xanchor='left')
            )
            
            fig.show()
    
    def display(self):
        """Display the complete interface"""
        self._update_param_visibility()
        self._update_plot()
        
        # Parameter controls box
        param_box = widgets.VBox([
            widgets.HTML('<h4>Function Parameters</h4>'),
            self.func_dropdown,
            self.param_description,
            self.param_a,
            self.param_b,
            self.param_c,
        ], layout=widgets.Layout(padding='10px', border='1px solid #ddd', margin='5px'))
        
        # Transformation controls box
        transform_box = widgets.VBox([
            widgets.HTML('<h4>Transformations</h4>'),
            widgets.HTML('<i>y = v_scale · f((x - h_shift) / h_scale) + v_shift</i>'),
            self.h_shift,
            self.v_shift,
            self.h_scale,
            self.v_scale,
        ], layout=widgets.Layout(padding='10px', border='1px solid #ddd', margin='5px'))
        
        # Cursor controls
        cursor_box = widgets.VBox([
            widgets.HTML('<h4>Cursor Control</h4>'),
            self.cursor_slider,
            self.cursor_info,
            self.inverse_button,
        ], layout=widgets.Layout(padding='10px', border='1px solid #ddd', margin='5px'))
        
        # Quiz box
        quiz_box = widgets.VBox([
            widgets.HTML('<h4>Properties Quiz</h4>'),
            widgets.HTML('<i>Select True for each property that applies:</i>'),
            self.check_symmetric,
            self.check_monotonic,
            self.check_convex,
            self.check_concave,
            self.check_nonnegative,
            widgets.HBox([self.check_button, self.reset_button]),
            self.quiz_feedback,
        ], layout=widgets.Layout(padding='10px', border='1px solid #ddd', margin='5px'))
        
        # Left panel (controls)
        left_panel = widgets.VBox([
            param_box,
            transform_box,
            cursor_box,
        ], layout=widgets.Layout(width='350px'))
        
        # Right panel (plot and quiz)
        right_panel = widgets.VBox([
            self.plot_output,
            quiz_box,
        ])
        
        # Main layout
        main_layout = widgets.HBox([left_panel, right_panel])
        
        display(main_layout)

In [ ]:
# Create and display the interactive visualization
explorer = FunctionExplorerVisualization()
explorer.display()